In [ ]:
%defaultDatasource jdbc:postgresql://

# コホートテーブルの作成　(my_cohort)

- 17歳より大きい
- 48時間以上滞在
- 出血がない

In [ ]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort ;
CREATE TABLE my_cohort AS
WITH co AS
(
    SELECT DISTINCT ON(icustays.subject_id, icustays.hadm_id) icustays.*, patients.gender, EXTRACT('year' from age(icustays.intime,patients.dob)) as age
    FROM icustays
    INNER JOIN patients 
      ON icustays.subject_id = patients.subject_id
    ORDER BY icustays.subject_id, icustays.hadm_id, icustays.intime, icustays.icustay_id DESC
),
bleed AS (
        select co.hadm_id
        from co
        where 
            co.age > 17 AND co.age < 200
            AND co.los >= 2.0
    EXCEPT
        select distinct hadm_id
        from diagnoses_icd as icd
        where 
            icd.icd9_code in('5310', '5312', '5314', '5316', '5320', '5322', '5324', '5326', '5330', '5332', '5334', '5336', '5340', '5342', '5344', '5346', '5780', '5781', '5789', '5693')
)
select co.*
from co, bleed
where co.hadm_id = bleed.hadm_id


In [ ]:
# t1(一番最初のヘモグロビンの測定)つきコホートテーブルの作成　(my_cohort_t1)

In [ ]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort_t1 ;
CREATE TABLE my_cohort_t1 AS
WITH first AS
(
select MIN(labevents.charttime) as first, labevents.hadm_id
from labevents, my_cohort as co
where 
    co.hadm_id = labevents.hadm_id
    AND co.intime < labevents.charttime
    AND co.outtime > labevents.charttime
    AND labevents.itemid = 51222
GROUP BY (labevents.hadm_id)
)
select co.*, first.first, labevents.value
from my_cohort as co, first, labevents
where co.hadm_id = first.hadm_id
AND first.hadm_id = labevents.hadm_id
AND labevents.itemid = 51222
AND labevents.charttime = first.first

一番最初のヘモグロビンの測定と、24時間後で一番早いヘモグロビンの測定

In [ ]:
set search_path to mimiciii ;
DROP TABLE IF EXISTS  my_cohort_t1_t2 ;
CREATE TABLE my_cohort_t1_t2 AS
WITH t2 AS
(
select MIN(labevents.charttime) as t2, labevents.hadm_id
from labevents, my_cohort_t1 as co
where
    co.hadm_id = labevents.hadm_id
    AND co.first  + interval '24 hours'< labevents.charttime
    AND co.outtime > labevents.charttime
    AND labevents.itemid = 51222
GROUP BY (labevents.hadm_id)
)
select co.*,t2.t2,labevents.value as t2value
from t2, my_cohort_t1 as co, labevents
where co.hadm_id = t2.hadm_id
AND t2.hadm_id = labevents.hadm_id
AND labevents.itemid = 51222
AND labevents.charttime = t2.t2

In [ ]:
set search_path to mimiciii ;
select * from my_cohort_t1_t2 limit 100